In [58]:
import re
import os
import numpy as np
import pandas as pd
from konlpy.tag import Mecab
import matplotlib.pylab as plt

plt.style.use('seaborn')
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]
pd.set_option('max_columns', 100) # So we can see more columns

In [145]:
def build_vocab(sentence):
    vocab = {}
    for word in sentence:
        try:
            vocab[word] += 1
        except KeyError:
            vocab[word] = 1
    return vocab

In [19]:
from soynlp.hangle import decompose

doublespace_pattern = re.compile('\s+')

def preprocess_text(sent):
    sent = sent.replace('XXX', 'X')
    def transform(char):
        if char == ' ':
            return char
        elif char == 'X':
            return ' X'
        elif char == '.':
            return '. '
        else:
            return char
    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_

In [315]:
!ls -alF ../input

total 933576
drwxr-xr-x 3 chrisyang chrisyang      4096  1월  9 22:38 ./
drwxrwxr-x 9 chrisyang chrisyang      4096  1월  9 13:08 ../
-rw-rw-r-- 1 chrisyang chrisyang  35034396  1월  8 02:40 fasttext_4gram_vocab.pkl
-rw-rw-r-- 1 chrisyang chrisyang   7494606  1월  9 20:57 fasttext_5gram_200_epcoh_10_nouns.pkl
-rw-rw-r-- 1 chrisyang chrisyang  50629687  1월  8 16:07 fasttext_bigram_300_epoch_10.pkl
-rw-rw-r-- 1 chrisyang chrisyang  50629687  1월  8 21:53 fasttext_bigram_300_epoch_30.pkl
-rw-r--r-- 1 chrisyang chrisyang  35034396  1월  7 14:38 fasttext_bigram_vocab.pkl
-rw-rw-r-- 1 chrisyang chrisyang  35034396  1월  8 02:39 fasttext_trigram_vocab.pkl
-rw-r--r-- 1 chrisyang chrisyang  35034396  1월  5 22:02 fasttext_vocab.pkl
-rw-rw-r-- 1 chrisyang chrisyang  14355335  1월  9 19:46 glove_200_epoch_10_nouns.pkl
-rw-rw-r-- 1 chrisyang chrisyang  97296415  1월  8 22:40 glove_300_epoch_10.pkl
drwxr-xr-x 2 chrisyang chrisyang      4096  1월  8 11:10 .ipynb_checkpoints/
-rw-rw-r-- 1 chrisyang chrisyang   

In [3]:
DATASET_PATH = '../input'

In [291]:
train_df = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATASET_PATH, 'public_test.csv'))
# train_df = pd.read_csv(os.path.join(DATASET_PATH, 'preprocessed_train.csv'))
# test_df = pd.read_csv(os.path.join(DATASET_PATH, 'preprocessed_test.csv'))
# submission = pd.read_csv(os.path.join(DATASET_PATH, 'submission.csv'))

In [169]:
mecab = Mecab()

In [14]:
for i, text in enumerate(train_df['text']):
    print(i, text)
    print()
    print(i, mecab.nouns(text))
    print()
    
    if i == 5: break

0  X은행성산 X팀장입니다. 행복한주말되세요

0 ['행성', '산', '팀장', '행복', '주말']

1 오늘도많이웃으시는하루시작하세요 X은행 진월동VIP라운지 X올림

1 ['오늘', '하루', '시작', '은행', '진월동', '라운지']

2 안녕하십니까 고객님. X은행입니다. 금일 납부하셔야 할 금액은 153600원 입니다. 감사합니다. 새해 복 많이 받으십시오. X은행옥포 X올림

2 ['안녕', '고객', '은행', '금일', '납부', '금액', '원', '감사', '새해', '복', '은행', '옥포']

3  X 고객님안녕하세요 X은행 X지점입니다지난 한 해 동안 저희 X지점에 보내주신 성원에 감사드립니다. 설렘으로 시작한 2017년소망하시는 일 모두 이 X 고객님의 가정에 늘 건강과 행복이 함께하길 기원하겠습니다. 사랑하는 가족과 함께 정을 나누는 행복한 설 명절 보내세요 X은행 X지점직원일동

3 ['고객', '안녕', '은행', '지점', '해', '동안', '저희', '지점', '성원', '감사', '시작', '년', '소망', '일', '고객', '가정', '건강', '행복', '기원', '사랑', '가족', '정', '행복', '설', '명절', '은행', '지점', '직원', '일동']

4 1월은 새로움이 가득 X입니다. 올 한해 더 많이행복한 한해되시길바랍니다

4 ['월', '한', '행복', '해']

5 행복한주말보내세요 X용현남전담직원대리 X올림

5 ['행복', '주말', '현남', '전담', '직원', '대리']



In [284]:
train_df['nouns'] = train_df['text'].apply(lambda x: mecab.nouns(preprocess_text(x)))
test_df['nouns'] = test_df['text'].apply(lambda x: mecab.nouns(preprocess_text(x)))

In [172]:
train_df['nouns']

0                                       [행성, 산, 팀장, 행복, 주말]
1                                [오늘, 하루, 시작, 은행, 진월동, 라운지]
2            [안녕, 고객, 은행, 금일, 납부, 금액, 원, 감사, 새해, 복, 은행, 옥포]
3         [고객, 안녕, 은행, 지점, 해, 동안, 저희, 지점, 성원, 감사, 시작, 년,...
4                                             [월, 한, 행복, 해]
                                ...                        
295940    [고객, 년, 뒤안길, 신년, 기운, 시점, 년, 고객, 가정, 건강, 행복, 원,...
295941                [고객, 실버, 웰빙, 신탁, 만기, 도래, 예정, 남양주, 지점]
295942    [해, 동안, 은행, 감사, 년, 기, 해년, 가족, 건강, 행복, 기원, 새해, ...
295943    [금, 예금, 적금, 만기, 예금, 시간, 내, 내점, 적금, 년, 자동, 연장, ...
295944    [안녕, 고객, 스타, 링크, 전담, 직원, 스타, 링크, 고객, 대상, 이벤트, ...
Name: nouns, Length: 295945, dtype: object

In [173]:
test_df['nouns']

0       [고객, 시작, 주, 행복, 기원, 지난주, 글로벌, 증시, 파월, 미, 연준, 의...
1       [긴급, 안내, 은행, 가락동, 지점, 헬리오, 기본, 대출, 이자, 조합, 납부,...
2       [고객, 안녕, 올해, 미세먼지, 유난, 거, 새해, 시작, 거, 설, 명절, 감사...
3       [고객, 행운, 이번, 기원, 날, 기원, 은행, 내당동, 종합, 금융, 센터, 팀...
4       [고객, 새해, 복, 은행, 코스트코, 퇴직, 연금, 담당자, 고객, 퇴직, 연금,...
                              ...                        
1621    [광고, 고객, 안녕, 고객, 업무, 처리, 창원, 중앙동, 지점, 계장, 업무, ...
1622    [고객, 저희, 지점, 거래, 감사, 본점, 서비스, 설문, 전화, 동의, 부탁, ...
1623    [고객, 저희, 은행, 지점, 이용, 감사, 이번, 주도, 행복, 시간, 기원, 은...
1624    [광고, 신청, 현금, 만, 원, 부동산, 리브, 지금, 부동산, 리브, 신청, 만...
1625    [광고, 고객, 안녕, 봉, 점, 대리, 군인, 연금, 수급, 대출, 상품, 출시,...
Name: nouns, Length: 1626, dtype: object

In [179]:
# build vocabulary
train_vocab = {}
for i in range(len(train_df)):
    sentence = train_df['nouns'][i]
    for word in sentence:
        try: 
            train_vocab[word] += 1
        except KeyError:
            train_vocab[word] = 1

test_vocab = {}
for i in range(len(test_df)):
    sentence = test_df['nouns'][i]
    for word in sentence:
        try: 
            test_vocab[word] += 1
        except KeyError:
            test_vocab[word] = 1

In [184]:
train_freq_one_words_dict = dict((k, v) for k, v in train_vocab.items() if v <= 1)
test_freq_one_words_dict = dict((k, v) for k, v in test_vocab.items() if v <= 1)

In [187]:
# 전체 vocab 갯수
len(train_vocab), len(test_vocab)

(23584, 4852)

In [185]:
# 고유값이 하나인 단어의 갯수
len(train_freq_one_words_dict), len(test_freq_one_words_dict)

(6327, 1590)

In [188]:
print("test words not in train {} (oov)".format(len(set(test_vocab.keys()) - set(train_vocab.keys()))))

test words not in train 572 (oov)


In [191]:
print("train total {} freq_one {} ratio {}".format(len(train_vocab), len(train_freq_one_words_dict), np.round(len(train_freq_one_words_dict) / len(train_vocab), 2)))
print("test total {} freq_one {} ratio {}".format(len(test_vocab), len(test_freq_one_words_dict), np.round(len(test_freq_one_words_dict) / len(test_vocab), 2)))

train total 23584 freq_one 6327 ratio 0.27
test total 4852 freq_one 1590 ratio 0.33


In [ ]:
# for word in train_freq_one_words_dict.keys():
#     del train_nouns_vocab[word]

# for word in test_freq_one_words_dict.keys():
#     del test_nouns_vocab[word]

# len(test_nouns_vocab)

In [192]:
# def remove_freq_one_word(words, freq_one_key):
#     for word in words:
#         if word in freq_one_key:
#             words.remove(word)
#     return words
        

# test_df['nouns_preprocessed'] = test_df['nouns'].apply(lambda x: remove_freq_one_word(x, test_freq_one_words_dict.keys()))

In [285]:
train_df['nouns'] = train_df['nouns'].apply(lambda x: ' '.join(x))
test_df['nouns'] = test_df['nouns'].apply(lambda x: ' '.join(x))

In [286]:
preprocessed_train_df = train_df[['id', 'year_month', 'nouns', 'smishing']]
preprocessed_test_df = test_df[['id', 'year_month', 'nouns']]

In [287]:
preprocessed_train_df

,id,year_month,nouns,smishing
0,0,2017-01,행성 산 팀장 행복 주말,0
1,1,2017-01,오늘 하루 시작 은행 진월동 라운지,0
2,2,2017-01,안녕 고객 은행 금일 납부 금액 원 감사 새해 복 은행 옥포,0
3,4,2017-01,고객 안녕 은행 지점 해 동안 저희 지점 성원 감사 시작 년 소망 일 고객 가정 건...,0
4,5,2017-01,월 한 행복 해,0
...,...,...,...,...
295940,336373,2018-12,고객 년 뒤안길 신년 기운 시점 년 고객 가정 건강 행복 원 일 해 새해 복 감사 은행,0
295941,336375,2018-12,고객 실버 웰빙 신탁 만기 도래 예정 남양주 지점,0
295942,336376,2018-12,해 동안 은행 감사 년 기 해년 가족 건강 행복 기원 새해 복 은행 점,0
295943,336377,2018-12,금 예금 적금 만기 예금 시간 내 내점 적금 년 자동 연장 은행 탄현 지점,0


In [288]:
preprocessed_train_df.to_csv('../input/preprocessed_train_nouns.csv', index=False)
preprocessed_test_df.to_csv('../input/preprocessed_test_nouns.csv', index=False)